In [8]:
import os
import json
import csv
import logging
from datetime import datetime
import requests
from typing import Dict, List, Optional, Union, Tuple, Any
from enum import Enum

class DataType(Enum):
    """Enum for different types of data being fetched"""
    VIEW = "view"
    LIKE = "like"
    INSPIRE = "inspire"
    RATE = "rate"
    SUMMARY = "summary"
    USER = "user"

    @property
    def is_paginated(self) -> bool:
        """Determine if the endpoint supports pagination"""
        return True  

class APIConfig:
    """Configuration class for API endpoints and parameters"""
    BASE_URL = "https://api.socialverseapp.com"
    RESONANCE_ALGORITHM = "resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
    FLIC_TOKEN = "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"
    PAGE_SIZE = 1000

    @classmethod
    def get_endpoint(cls, data_type: DataType) -> Tuple[str, Dict, Dict]:
        """Get API endpoint and parameters for a given data type"""
        base_params = {"page": 1, "page_size": cls.PAGE_SIZE}
        base_headers = {}
        
        endpoints = {
            DataType.VIEW: ("/posts/view", {"resonance_algorithm": cls.RESONANCE_ALGORITHM}, {}),
            DataType.LIKE: ("/posts/like", {"resonance_algorithm": cls.RESONANCE_ALGORITHM}, {}),
            DataType.INSPIRE: ("/posts/inspire", {"resonance_algorithm": cls.RESONANCE_ALGORITHM}, {}),
            DataType.RATE: ("/posts/rating", {"resonance_algorithm": cls.RESONANCE_ALGORITHM}, {}),
            DataType.SUMMARY: ("/posts/summary/get", {}, {"Flic-Token": cls.FLIC_TOKEN}),
            DataType.USER: ("/users/get_all", {}, {"Flic-Token": cls.FLIC_TOKEN})
        }
        
        endpoint, extra_params, extra_headers = endpoints[data_type]
        params = {**base_params, **extra_params}
        headers = {**base_headers, **extra_headers}
        
        return f"{cls.BASE_URL}{endpoint}", params, headers

class APIDataFetcher:
    """A comprehensive utility for fetching paginated data from APIs with advanced features."""
    
    def __init__(self, base_log_dir: str = 'api_logs'):
        """Initialize the APIDataFetcher with logging and storage configurations."""
        self.base_log_dir = base_log_dir
        self._setup_directories()
        self._setup_logging()
    
    def _setup_directories(self):
        """Set up necessary directories for logs and data storage"""
        os.makedirs(self.base_log_dir, exist_ok=True)
        os.makedirs(os.path.join(self.base_log_dir, 'json'), exist_ok=True)
        os.makedirs(os.path.join(self.base_log_dir, 'csv'), exist_ok=True)
        os.makedirs(os.path.join(self.base_log_dir, 'logs'), exist_ok=True)
    
    def _setup_logging(self):
        """Configure logging with both file and console handlers"""
        log_file = os.path.join(self.base_log_dir, 'logs', f'api_fetch_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s: %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def fetch_data(
        self,
        data_type: DataType,
        export_csv: bool = True
    ) -> List[Dict]:
        """
        Fetch data from an API with comprehensive error handling and logging.
        
        :param data_type: Type of data to fetch (from DataType enum)
        :param export_csv: Whether to export data to CSV
        :return: List of data items
        """
        api_url, params, headers = APIConfig.get_endpoint(data_type)
        
        self.logger.info(f"Starting data fetch from: {api_url}")
        
        try:
            data = self._fetch_paginated_data(api_url, params, headers, data_type)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            json_filename = f"{data_type.value}.json"
            csv_filename = f"{data_type.value}.csv"
            
            json_path = os.path.join(self.base_log_dir, 'json', json_filename)
            csv_path = os.path.join(self.base_log_dir, 'csv', csv_filename)
            

            os.makedirs(os.path.join(self.base_log_dir, 'json'), exist_ok=True)
            os.makedirs(os.path.join(self.base_log_dir, 'csv'), exist_ok=True)
            
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            self.logger.info(f"JSON data stored at: {json_path}")
            
            if export_csv and data:
                if data: 
                    try:
                        fieldnames = list(data[0].keys())
                        with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                            writer.writeheader()
                            writer.writerows(data)
                        self.logger.info(f"CSV data stored at: {csv_path}")
                    except Exception as csv_error:
                        self.logger.error(f"Error writing CSV: {csv_error}")
            
            self.logger.info(f"Completed data fetch. Total items retrieved: {len(data)}")
            
            return data
            
        except Exception as e:
            self.logger.error(f"Error during data fetch: {e}")
            return []
    
    def _fetch_paginated_data(
        self,
        api_url: str,
        params: Dict,
        headers: Dict,
        data_type: DataType
    ) -> List[Dict]:
        """Fetch data from paginated endpoints"""
        all_data = []
        page = 1
        
        while True:
            params['page'] = page
            response = self._make_api_request(api_url, params, headers)
            
            if data_type in [DataType.VIEW, DataType.LIKE, DataType.INSPIRE, DataType.RATE]:
                page_data = response.get('posts', [])
            elif data_type == DataType.SUMMARY:
                page_data = response.get('posts', [])
            elif data_type == DataType.USER:
                page_data = response.get('users', [])
            else:
                page_data = []
            
            if not page_data:
                break
                
            all_data.extend(page_data)
            self.logger.info(f"Page {page}: Retrieved {len(page_data)} items")
            
            if len(page_data) < APIConfig.PAGE_SIZE:
                break
                
            page += 1
        
        return all_data
    
    def _make_api_request(self, url: str, params: Dict, headers: Dict) -> Dict:
        """Make API request with error handling"""
        try:
            response = requests.get(url, params=params, headers=headers, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"API Request Error: {e}")
        except json.JSONDecodeError as e:
            raise Exception(f"JSON Parsing Error: {e}")

def main():
    """Example usage of the APIDataFetcher"""
    fetcher = APIDataFetcher()

    all_data = {}
    
    for data_type in DataType:
        data = fetcher.fetch_data(data_type)
        all_data[data_type.value] = data
        
        print(f"Retrieved {len(data)} {data_type.value} items.")
    
    print("\nData Lengths:")
    for data_type, data in all_data.items():
        if isinstance(data, list):
            print(f"{data_type}: {len(data)} items")
        else:
            print(f"{data_type}: 1 item")

if __name__ == "__main__":
    main()

2024-12-12 14:56:10,525 - INFO: Starting data fetch from: https://api.socialverseapp.com/posts/view
2024-12-12 14:56:11,546 - INFO: Page 1: Retrieved 1000 items
2024-12-12 14:56:12,558 - INFO: Page 2: Retrieved 1000 items
2024-12-12 14:56:14,705 - INFO: Page 3: Retrieved 1000 items
2024-12-12 14:56:16,369 - INFO: Page 4: Retrieved 1000 items
2024-12-12 14:56:18,906 - INFO: Page 5: Retrieved 1000 items
2024-12-12 14:56:19,992 - INFO: Page 6: Retrieved 1000 items
2024-12-12 14:56:24,599 - INFO: Page 7: Retrieved 949 items
2024-12-12 14:56:24,637 - INFO: JSON data stored at: api_logs/json/view.json
2024-12-12 14:56:24,647 - INFO: CSV data stored at: api_logs/csv/view.csv
2024-12-12 14:56:24,648 - INFO: Completed data fetch. Total items retrieved: 6949
2024-12-12 14:56:24,649 - INFO: Starting data fetch from: https://api.socialverseapp.com/posts/like


Retrieved 6949 view items.


2024-12-12 14:56:25,461 - INFO: Page 1: Retrieved 1000 items
2024-12-12 14:56:26,634 - INFO: Page 2: Retrieved 279 items
2024-12-12 14:56:26,640 - INFO: JSON data stored at: api_logs/json/like.json
2024-12-12 14:56:26,642 - INFO: CSV data stored at: api_logs/csv/like.csv
2024-12-12 14:56:26,643 - INFO: Completed data fetch. Total items retrieved: 1279
2024-12-12 14:56:26,643 - INFO: Starting data fetch from: https://api.socialverseapp.com/posts/inspire


Retrieved 1279 like items.


2024-12-12 14:56:28,786 - INFO: Page 1: Retrieved 268 items
2024-12-12 14:56:28,788 - INFO: JSON data stored at: api_logs/json/inspire.json
2024-12-12 14:56:28,789 - INFO: CSV data stored at: api_logs/csv/inspire.csv
2024-12-12 14:56:28,789 - INFO: Completed data fetch. Total items retrieved: 268
2024-12-12 14:56:28,790 - INFO: Starting data fetch from: https://api.socialverseapp.com/posts/rating


...

Retrieved 2907 rate items.


2024-12-12 14:56:37,150 - INFO: Page 1: Retrieved 1000 items
2024-12-12 14:56:40,125 - INFO: Page 2: Retrieved 857 items
2024-12-12 14:56:40,441 - INFO: JSON data stored at: api_logs/json/summary.json
2024-12-12 14:56:40,710 - INFO: CSV data stored at: api_logs/csv/summary.csv
2024-12-12 14:56:40,711 - INFO: Completed data fetch. Total items retrieved: 1857
2024-12-12 14:56:40,711 - INFO: Starting data fetch from: https://api.socialverseapp.com/users/get_all


Retrieved 1857 summary items.


2024-12-12 14:56:42,649 - INFO: Page 1: Retrieved 1000 items
2024-12-12 14:56:44,421 - INFO: Page 2: Retrieved 306 items
2024-12-12 14:56:44,464 - INFO: JSON data stored at: api_logs/json/user.json
2024-12-12 14:56:44,476 - INFO: CSV data stored at: api_logs/csv/user.csv
2024-12-12 14:56:44,477 - INFO: Completed data fetch. Total items retrieved: 1306


Retrieved 1306 user items.

Data Lengths:
view: 6949 items
like: 1279 items
inspire: 268 items
rate: 2907 items
summary: 1857 items
user: 1306 items
